In [ ]:
#Program to add speaker and filename column in each word file
import os
import sys
import glob
import os.path
import pandas as pd
import re

list_of_files = glob.glob(r'D:\Jay\columbia-games-corpus\data\**\*.words',recursive=True) 

for file_name in list_of_files:    
    out_name=file_name+'_with_speaker'
    csv_input = pd.read_csv(file_name,delimiter=' ',header=None)
    csv_input['Speaker'] = file_name[-7]
    csv_input['filename'] = file_name
    csv_input.to_csv(out_name, index=False,header=False,sep=' ')

In [ ]:
#Program to merge Speaker text files and convert flac files
import os
import fnmatch
import sys
import subprocess

OLD_BASE = r'D:\Jay\columbia-games-corpus\data'
NEW_BASE = r'D:\Jay\columbia-games-corpus\jkdata'

def merge_files(infiles):
    output=os.path.basename(infiles[0][:-9])+'txt'
    outfiletxt = os.path.join(new_dir, output)
    with open(outfiletxt, 'wb') as fo:
        for infile in infiles:
            with open(infile, 'rb') as fi:
                fo.write(fi.read())

def convert_files(infiles):
    output_channel1=os.path.basename(infiles[0][:-5])+'.wav'
    output_channel2=os.path.basename(infiles[1][:-5])+'.wav'
    outfile1 = os.path.join(new_dir, output_channel1)
    outfile2 = os.path.join(new_dir, output_channel2)
    ffp=r"C:\ffmpeg-master-latest-win64-gpl\bin\ffmpeg"
    cmd2wav1 = ffp+' -i ' + infiles[0] + ' ' + outfile1
    cmd2wav2 = ffp+' -i ' + infiles[1] + ' ' + outfile2
    #print(cmd2wav)
    subprocess.call(cmd2wav1, shell=True)
    subprocess.call(cmd2wav2, shell=True)

for (dirpath, dirnames, filenames) in os.walk(OLD_BASE):
    base, tail = os.path.split(dirpath)
    if base != OLD_BASE: continue  # Don't operate on OLD_BASE, only children directories

    # Build infiles list for flac objects
    object_flac = sorted([os.path.join(dirpath, filename) for filename in filenames if fnmatch.fnmatch(filename,"*objects*.flac")])
    # Build infiles list for words objects
    object_text = sorted([os.path.join(dirpath, filename) for filename in filenames if fnmatch.fnmatch(filename,"*objects*.words_with_speaker")])

    # Create output directory
    new_dir =  os.path.join(NEW_BASE, tail)
    os.mkdir(new_dir)  # This will raise an OSError if the directory already exists    

    # Merge
    convert_files(object_flac)
    merge_files(object_text)




In [ ]:
#Program to extract turns
import os
import sys
import glob
import os.path
import pandas as pd

list_of_files = glob.glob(r'D:\Jay\columbia-games-corpus\jkdata\**\*.words_wittxt',recursive=True) 

for file_name in list_of_files:    
    out_name=file_name[:-14]+'txt'
    headers= ['start','end','words','Speaker','filename']
    csv_input = pd.read_csv(file_name,delimiter=' ')
    csv_input.columns = headers
    #csv_input
    cgc2=csv_input.sort_values(['start'])    
    cgc3=cgc2[cgc2["words"].str.contains("#")==False]
    cgc3[['start', 'end']] = cgc3[['start', 'end']].astype(str)
    cgc3['obj1_count'] = (cgc3['Speaker'].ne(cgc3['Speaker'].shift())).cumsum()
    df3=cgc3.groupby('obj1_count').agg(lambda x: ' '.join(x))
    df3['start'] = df3['start'].map(lambda x: x.split(" ")[0])
    df3['end'] = df3['end'].map(lambda x: x.split(" ")[-1])
    df3['Speaker'] = df3['Speaker'].map(lambda x: x.split(" ")[-1])
    df3['filename'] = df3['filename'].map(lambda x: x.split(" ")[-1])
    df3['filename'] = df3['filename'].str.replace('words','flac')
    df3['session']=re.search('(\d\d)',file_name)[0]
    if df3.iloc[-1]['Speaker'] == df3.iloc[0]['Speaker']:
        df4=df3.drop(df3.index[len(df3)-1])
        df4.to_csv(out_name, index=False,header=False,sep='\t')
        del df4
    else:
        df3.to_csv(out_name, index=False,header=False,sep='\t')
    del csv_input,cgc2,cgc3,df3

In [ ]:
import glob

#Delete files with unwanted extension
for f in glob.glob(r'D:\Jay\columbia-games-corpus\jkdata\**\*.words_wittxt',recursive=True):
    os.remove(f)

In [3]:
#Program to extract embeddings
import pandas as pd
from sentence_transformers import SentenceTransformer, util
from transformers import BertTokenizer, BertModel
import torch
from functools import reduce
import csv
from sklearn.metrics.pairwise import cosine_similarity
import random
import subprocess
import re
import glob
import os,sys
import numpy as np
import soundfile as sf
import wave
import json
import tensorflow as tf1
import tensorflow_hub as hub
# Import TF 2.X and make sure we're running eager.
import tensorflow.compat.v2 as tf
#import tensorflow as tf
from sklearn.metrics.pairwise import cosine_similarity
modulev3=None
modulev3_graph=None
import pickle
tf.enable_v2_behavior()
assert tf.executing_eagerly()
   
def get_TRILLv3_signal(signal,samplerate):
    global modulev3
    if modulev3==None:
        print('******************\nLoading model ...\n******************')    
        modulev3 = hub.load(r'D:\Jay\columbia-games-corpus\trill_extraction_v2\v3')
    
    
    max_int16 = 2**15
    chunks_cnt=int(signal.shape[0]/(samplerate*10.0))#10 seconds max in chunk
    if chunks_cnt==0:
        chunks=[signal]
    else:
        chunks=np.array_split(signal, chunks_cnt)
    
    trillv3_emb_all=np.empty(shape=(0,512))
    
    for chunk in chunks:
        trillv3 = modulev3(samples=chunk, sample_rate=samplerate)
        trillv3_emb = trillv3['embedding']
        trillv3_emb_all=np.concatenate((trillv3_emb_all, trillv3_emb))

    trillv3_emb_avg = np.mean(trillv3_emb_all, axis=0, keepdims=False)

    return (trillv3_emb_avg.tolist())    

def check_wav_format(wav_file, start, end):
    wf = wave.open(wav_file)
    nchannels, sampwidth, framerate, nframes, comptype, compname = wf.getparams()
    print(nchannels, sampwidth, framerate, nframes, comptype, compname)
    wav_length = float(nframes) / float(framerate)
    print(wav_length)
    if nchannels!=1:
        print('Error: Incoming audio file has more then 1 channel')
        return(-1) 
				
    if framerate!=16000:
        print('Error: Incoming audio file sampling frequency must be 16000')
        return(-2)
				
    if sampwidth!=2:
        print('Error: Incoming audio file sample width must be 16 bit')
        return(-3)

    if wav_length<end:
        print('Error: The duration of the audio file is shorter than the required end time')
        return(-4)
		
    if start<0.0:
        print('Error: start time is lower then 0.0')
        return(-5)
    
    if start>=end:
        print('Error: start time is larger then end time')
        return(-6)    
		
				
    return(framerate)

def get_TRILLv3_audiofile_from_to(wav_file,start,end):
    print('get_TRILLv3_signal:',wav_file,start,end)
    samplerate=check_wav_format(wav_file, start, end)
    if samplerate<0:
        return(null)    
    startsample=int(start*samplerate)
    endsample=int(end*samplerate)
    signal, samplerate = sf.read(wav_file,start=startsample, stop=endsample)
    print(len(signal),samplerate)    

    trill=get_TRILLv3_signal(signal,samplerate)
    return(trill)

audio_embedding_path = 'D:\\Jay\\columbia-games-corpus\\jkdata\\embedding\\audio\\'
text_embedding_path = 'D:\\Jay\\columbia-games-corpus\\jkdata\\embedding\\text\\'
path = 'D:\\Jay\\\columbia-games-corpus\\jkdata\\'
model = SentenceTransformer("sentence-transformers/msmarco-distilbert-base-v4")
all_files = os.listdir(path)

for root, dirs, files in os.walk(path):
    for file in files:
        if file.endswith('.txt'):
            with open(os.path.join(root, file), 'r', encoding="utf8") as f:
                out_audio_embedding= os.path.join(audio_embedding_path, os.path.basename(file))
                out_text_embedding= os.path.join(text_embedding_path, os.path.basename(file))
                sentenceembeddings=[]
                audioembeddings = []
                sentence_embeddings_tensor = []
                text = f.readlines()
                for line in text:
                    n1=line.split('\t')[0]
                    n2=line.split('\t')[1]
                    n3=line.split('\t')[4].replace('flac', 'wav').replace('data', 'jkdata')
                    sentence=line.split('\t')[2]
                    x=get_TRILLv3_audiofile_from_to(n3,float(n1),float(n2))
                    sentenceembeddings.append(sentence)
                    audioembeddings.append(x)
                sentence_embeddings_tensor = model.encode(sentenceembeddings, convert_to_tensor=False)
                sentence_embeddings_tensor=sentence_embeddings_tensor.tolist()
                with open(out_audio_embedding+'.pkl', 'wb') as f:
                    pickle.dump(audioembeddings, f)
                with open(out_text_embedding+'.pkl', 'wb') as f:
                    pickle.dump(sentence_embeddings_tensor, f)

In [1]:
#Program to measure distance
import pandas as pd
import csv
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import util
import random
import subprocess
import re
import glob
import os,sys
import numpy as np
import pickle

random.seed(223)
#Define path of embedding files and output
audio_embedding_path = 'D:\\Jay\\columbia-games-corpus\\jkdata\\embedding\\audio'
text_embedding_path = 'D:\\Jay\\columbia-games-corpus\\jkdata\\embedding\\text'
output_path = 'D:\\Jay\\columbia-games-corpus\\jkdata\\output\\'

for audiofile,textfile in zip(os.listdir(audio_embedding_path),os.listdir(text_embedding_path)):
        if audiofile.endswith(".pkl") & textfile.endswith(".pkl"):
            current_audiofile = os.path.join(audio_embedding_path,audiofile)
            current_textfile = os.path.join(text_embedding_path,textfile)
            output_filename= os.path.splitext(os.path.join(output_path, os.path.basename(audiofile)))[0]
            #Open both audio and semantic pickle files
            with open(current_audiofile, 'rb') as a, open(current_textfile, 'rb') as b:
                audio_embeddings_tensor=pickle.load(a)
                sentence_embeddings_tensor=pickle.load(b)
                sentence_embedding_random = []
                audio_embedding_random = []
                text_speaker1=[]
                text_speaker2=[]
                audio_speaker1=[]
                audio_speaker2=[]
                same_pairs_text = []
                not_samepairs_text_array = []
                not_samepairs_text=[]
                same_pairs_audio = []
                not_samepairs_audio_array = []
                not_samepairs_audio=[]
                
                for i in range(0, len(sentence_embeddings_tensor)-1):
                    #Condition for random turns to exclude current and next turn 
                    sentence_embedding_random = [item for item in sentence_embeddings_tensor if np.logical_and(
                                                 item!=sentence_embeddings_tensor[i] , item != sentence_embeddings_tensor[i+1])]
                    audio_embedding_random = [item for item in audio_embeddings_tensor if np.logical_and(
                                                 item!=audio_embeddings_tensor[i] , item != audio_embeddings_tensor[i+1])]

                    if(i%2==0):
                        #Measure Partner distance for even turns
                        spt=util.cos_sim(sentence_embeddings_tensor[i],sentence_embeddings_tensor[i+1]).item()
                        same_pairs_text.append(spt)
                        spa=util.cos_sim(audio_embeddings_tensor[i],audio_embeddings_tensor[i+1]).item()
                        same_pairs_audio.append(spa)
                        
                        for x in range(10):
                            #Measure other distance for odd turns
                            nspta=util.cos_sim(sentence_embeddings_tensor[i],sentence_embedding_random[random.randrange(1,len(sentence_embedding_random),2)]).item()
                            not_samepairs_text_array.append(nspta)
                            nspaa=util.cos_sim(audio_embeddings_tensor[i],audio_embedding_random[random.randrange(1,len(audio_embedding_random),2)]).item()
                            not_samepairs_audio_array.append(nspaa)

                        nspt = np.mean(not_samepairs_text_array)
                        nspa = np.mean(not_samepairs_audio_array)

                        not_samepairs_text_array = []
                        not_samepairs_audio_array = []

                        not_samepairs_text.append(nspt)                       
                        not_samepairs_audio.append(nspa)

                    elif(i%2!=0):
                        #Measure Partner distance for odd turns
                        spt=util.cos_sim(sentence_embeddings_tensor[i],sentence_embeddings_tensor[i+1]).item()
                        same_pairs_text.append(spt)
                        spa=util.cos_sim(audio_embeddings_tensor[i],audio_embeddings_tensor[i+1]).item()
                        same_pairs_audio.append(spa)
                        
                        for x in range(10):
                            #Measure other distance for odd turns
                            nspta=util.cos_sim(sentence_embeddings_tensor[i],sentence_embedding_random[random.randrange(0,len(sentence_embedding_random),2)]).item()
                            not_samepairs_text_array.append(nspta)
                            nspaa=util.cos_sim(audio_embeddings_tensor[i],audio_embedding_random[random.randrange(0,len(audio_embedding_random),2)]).item()
                            not_samepairs_audio_array.append(nspaa)

                        nspt = np.mean(not_samepairs_text_array)
                        nspa = np.mean(not_samepairs_audio_array)

                        not_samepairs_text_array = []
                        not_samepairs_audio_array = []

                        not_samepairs_text.append(nspt)                       
                        not_samepairs_audio.append(nspa)

                for n in range(0, len(sentence_embeddings_tensor)-2):
                    #Measure self-distance
                    if(n%2==0):
                        same_spkr1_text=util.cos_sim(sentence_embeddings_tensor[n],sentence_embeddings_tensor[n+2]).item()
                        text_speaker1.append(same_spkr1_text)
                        same_spkr1_audio=util.cos_sim(audio_embeddings_tensor[n],audio_embeddings_tensor[n+2]).item()
                        audio_speaker1.append(same_spkr1_audio)
                        
                    elif(n%2!=0):
                        same_spkr2_text=util.cos_sim(sentence_embeddings_tensor[n],sentence_embeddings_tensor[n+2]).item()
                        text_speaker2.append(same_spkr2_text)
                        same_spkr2_audio=util.cos_sim(audio_embeddings_tensor[n],audio_embeddings_tensor[n+2]).item()
                        audio_speaker2.append(same_spkr2_audio)
                        
                df = pd.DataFrame(data=None)
                df['Same_pair_text'] = pd.Series(same_pairs_text)
                df['Not_Same_Pair_text'] = pd.Series(not_samepairs_text)
                df['Same_pair_audio'] = pd.Series(same_pairs_audio)
                df['Not_Same_Pair_audio'] = pd.Series(not_samepairs_audio)

                df['Speaker1_text_self_distance'] = pd.Series(text_speaker1)
                df['Speaker2_text_self_distance'] = pd.Series(text_speaker2)
                df['Speaker1_audio_self_distance'] = pd.Series(audio_speaker1)
                df['Speaker2_audio_self_distance'] = pd.Series(audio_speaker2)

                df.insert(0, 'Turn_number', range(1, 1 + len(df)))
                df.to_csv(output_filename, index=False,sep='\t')
                same_pairs_text = []
                not_samepairs_text_array = []
                not_samepairs_text=[]
                same_pairs_audio = []
                not_samepairs_audio_array = []
                not_samepairs_audio=[]
                sentence_embedding_random = []
                audio_embedding_random = []
                text_speaker1 = []
                text_speaker2 = []
                audio_speaker1 = []
                audio_speaker2 = []
                del df,spt,nspta,nspt,spa,nspaa,nspa,same_spkr1_text,same_spkr1_audio,same_spkr2_text,same_spkr2_audio

         

In [9]:
#Program to measure proximity, convergence and syncrony
import numpy as np
import pandas as pd
import scipy
import pingouin as pg
from scipy.stats import ttest_ind
from math import sqrt
import sys
from scipy import stats
import os
import sys
import glob
import os.path
import pandas as pd
from statsmodels.stats.multitest import multipletests
list_of_files = glob.glob(r'D:\Jay\columbia-games-corpus\jkdata\output\*.txt',recursive=True)
output_path = r'D:\Jay\columbia-games-corpus\jkdata\statsoutput'
for file_name in list_of_files:
    out_name= os.path.join(output_path, os.path.basename(file_name))
    csv_input = pd.read_csv(file_name, delimiter='\t')
    
    #For relationship between audio and text
    s,p = stats.pearsonr(csv_input['Same_pair_audio'], csv_input['Same_pair_text'])
    csv_input['Relationship-bw-both-Pearson rho'] = s
    csv_input['Relationship-bw-both-Pearson p-value'] = p
    csv_input['Relationship-bw-both-Pearson p-value_adj'] = csv_input['Relationship-bw-both-Pearson p-value'].lt(0.05/12)
  
    #Proximity
    proximity_text=stats.ttest_rel(csv_input['Same_pair_text'], csv_input['Not_Same_Pair_text'])
    csv_input['Proximity-t-value_text']= proximity_text.statistic
    csv_input['Proximity-p-value_text']= proximity_text.pvalue
    csv_input['Proximity-p-value_text_adj'] = csv_input['Proximity-p-value_text'].lt(0.05/12)


    proximity_audio=stats.ttest_rel(csv_input['Same_pair_audio'], csv_input['Not_Same_Pair_audio'])
    csv_input['Proximity-t-value_audio']= proximity_audio.statistic
    csv_input['Proximity-p-value_audio']= proximity_audio.pvalue
    csv_input['Proximity-p-value_audio_adj'] = csv_input['Proximity-p-value_audio'].lt(0.05/12)
    
    #Convergence
    convtextr,convtextp = stats.pearsonr(csv_input['Same_pair_text'], csv_input['Turn_number'])
    csv_input['Convergence-Pearson rho_text'] = convtextr
    csv_input['Convergence-Pearson p-value_text'] = convtextp
    csv_input['Convergence-Pearson p-value_text_adj'] = csv_input['Convergence-Pearson p-value_text'].lt(0.05/12)

    convaudior,convaudiop = stats.pearsonr(csv_input['Same_pair_audio'], csv_input['Turn_number'])
    csv_input['Convergence-Pearson rho_audio'] = convaudior
    csv_input['Convergence-Pearson p-value_audio'] = convaudiop
    csv_input['Convergence-Pearson p-value_audio_adj'] = csv_input['Convergence-Pearson p-value_audio'].lt(0.05/12)

    #Synchrony
    csv_input.dropna(inplace=True)
    synctextr,synctextp = stats.pearsonr(csv_input['Speaker1_text_self_distance'], csv_input['Speaker2_text_self_distance'])
    csv_input['Syncrony-Pearson rho_text'] = synctextr
    csv_input['Syncrony-Pearson p-value_text'] = synctextp
    csv_input['Syncrony-Pearson p-value_text_adj'] = csv_input['Syncrony-Pearson p-value_text'].lt(0.05/12)

    syncaudior,syncaudiop = stats.pearsonr(csv_input['Speaker1_audio_self_distance'], csv_input['Speaker2_audio_self_distance'])
    csv_input['Syncrony-Pearson rho_audio'] = syncaudior
    csv_input['Syncrony-Pearson p-value_audio'] = syncaudiop
    csv_input['Syncrony-Pearson p-value_audio_adj'] = csv_input['Syncrony-Pearson p-value_audio'].lt(0.05/12)
 

    csv_input.drop(['Same_pair_text', 'Not_Same_Pair_text', 'Same_pair_audio', 'Not_Same_Pair_audio',
                'Speaker1_text_self_distance','Speaker2_text_self_distance','Turn_number','Speaker1_audio_self_distance',
                'Speaker2_audio_self_distance'], axis=1,inplace=True)
    csv_input = csv_input.loc[[0]]
    csv_input.to_csv(out_name, index=False,sep='\t')
    del csv_input,s,p,proximity_text,convtextr,convtextp,synctextr,synctextp,proximity_audio,convaudior,convaudiop,syncaudior,syncaudiop

In [10]:
import pandas as pd
import glob
list_of_files = glob.glob(r'D:\Jay\columbia-games-corpus\jkdata\statsoutput\*.txt',recursive=True)
out_name = r'D:\Jay\columbia-games-corpus\jkdata\statsoutput\merged_stats.txt'
li = []

for filename in list_of_files:
    df = pd.read_csv(filename, index_col=None, header=0, delimiter='\t')
    basename = os.path.basename(filename)
    file_name = os.path.splitext(basename)[0]
    df['File_name'] = file_name
    #Re-arrange columns with filename as first column
    cols = df.columns.tolist()
    cols = cols[-1:] + cols[:-1]
    #Define new arranged column to data-frame
    df = df[cols]
    #Append the dataframe to list
    li.append(df)
frame = pd.concat(li, axis=0, ignore_index=True) 

frame.to_csv(out_name, index=False,sep='\t')